In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
from torchvision.transforms import ToPILImage

%cd /content

if os.path.exists('/content/hiwyn'):
  %rm -rf /content/hiwyn
!git clone https://github.com/ByeongHyunPak/hiwyn.git

%cd /content/hiwyn

In [ ]:
# opt variables
sd_version = '2.0'
negative = ''
steps = 50

In [ ]:
# view directions
theta_range = (0, 360)
phi_range = (-45, 45)

num_phi = 4
num_theta = [3, 6, 6, 3]
assert num_phi == len(num_theta)

directions = []
phis = np.linspace(*phi_range, num_phi, endpoint=True)
for i in range(num_phi):
    thetas = np.linspace(*theta_range, num_theta[i], endpoint=False)
    for theta in thetas:
        directions.append((theta, phis[i]))
    print(*directions[-num_theta[i]:])

In [ ]:
H, W = 1024, 2048 # ERP resolution
version = "0.0.0" # ERPDiffusion version
prompt = "Realistic cityscape of Florence"
dir_name = "imgs"

In [ ]:
if __name__ == '__main__':
    
    import gc
    import traceback
    from diffusions import *

    seed_everything(2024)

    ERPDiffusion = globals()[f"ERPDiffusion_{version.replace('.', '_')}"]

    base_dir = f'/content/{dir_name}'
    save_dir = f'{base_dir}/{prompt.split(" ")[0]}-{version}/'
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(save_dir, exist_ok=True)

    try:
        sd = ERPDiffusion(device=torch.device('cuda'), sd_version=sd_version, views=directions) 
        outputs = sd.text2erp(
            prompt, negative, height=H, width=W, num_inference_steps=steps, save_dir=save_dir)
        
        del outputs
        del sd
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
        
    except Exception:
        
        del sd
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.empty_cache()
       
        print(traceback.format_exc())